# BART (large-sized model), fine-tuned on CNN Daily Mail by Facebook

- [facebook/bart-large-cnn](https://huggingface.co/facebook/bart-large-cnn)

In [ ]:
!pip install transformers

In [ ]:
!pip install -U sagemaker

In [ ]:
!pip install -U boto3 awswrangler

In [ ]:
!python -m pip install boto3

In [7]:
import pandas as pd
import sagemaker
import boto3
import transformers
from transformers import pipeline
import awswrangler as wr
import json

In [8]:
print(transformers.__version__)

4.24.0


In [9]:
!pip3 show torch

Name: torch
Version: 1.10.2+cu113
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /opt/conda/lib/python3.8/site-packages
Requires: typing-extensions
Required-by: awsio, torchnet, torchvision


In [10]:
!python3 -V

Python 3.8.10


## Deploy model using Amazon SageMaker

In [30]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker

role = sagemaker.get_execution_role()
# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'facebook/bart-large-cnn',
	'HF_TASK':'summarization'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.17.0',
	pytorch_version='1.10.2',
	py_version='py38',
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge' # ec2 instance type
)

Exception ignored in: <function tqdm.__del__ at 0x7fb7fab9e700>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/tqdm/std.py", line 1147, in __del__
    self.close()
  File "/opt/conda/lib/python3.8/site-packages/tqdm/notebook.py", line 286, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm' object has no attribute 'disp'


------!

In [31]:
predictor.predict({
	'inputs': "The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."
})

[{'summary_text': 'The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world.'}]

In [75]:
!conda install -c conda-forge tqdm 

usage: conda [-h] [-V] command ...
conda: error: unrecognized arguments: --no-input


In [12]:
def p(self, b=None):
    if b is None:
        b = self.a
    print (b)

In [13]:
# Inputs longer than 1024 tokens cause irrecoverable CUDA errors on
# SageMaker. Make sure that each text is at most 1024 tokens.
inputs = self.tokenizer(texts, max_length=1024, padding="longest",
                        truncation=True)
truncated_texts = [self.tokenizer.decode(i, skip_special_tokens=True, clean_up_tokenization_spaces=False)
                   for i in inputs["input_ids"]]
output = predictor.predict({"inputs": truncated_texts, "parameters": parameters})
summaries = [summary["summary_text"] for summary in output]

NameError: name 'self' is not defined

In [11]:
from transformers import BartForConditionalGeneration, BartTokenizer
from typing import List
from tqdm import tqdm

def old_summarization_pipeline(text: List[str]) -> List[str]:
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
    input_ids = tokenizer.batch_encode_plus(text, return_tensors='pt', max_length=1024)['input_ids']
    summary_ids = model.generate(input_ids)
    summaries = [tokenizer.decode(s) for s in summary_ids]
    return summaries

In [12]:
text = '=' * 10257  
old_summarization_pipeline(text)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [80]:
def old_summarization_pipeline(text: List[str]) -> List[str]:
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
    input_ids = tokenizer.batch_encode_plus(text, truncation=True, padding=True, return_tensors='pt', max_length=1024)['input_ids']
    summary_ids = model.generate(input_ids)
    summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=False) for s in summary_ids]
    return summaries

In [81]:
text = '''Hawaii comprises nearly the entire Hawaiian archipelago, 137 volcanic islands spanning 1,500 miles (2,400 km) that are physiographically and ethnologically part of the Polynesian subregion of Oceania. The state's ocean coastline is consequently the fourth-longest in the U.S., at about 750 miles (1,210 km).[b] The eight main islands, from northwest to southeast, are Niʻihau, Kauaʻi, Oʻahu, Molokaʻi, Lānaʻi, Kahoʻolawe, Maui, and Hawaiʻi—the last of these, after which the state is named, is often called the "Big Island" or "Hawaii Island" to avoid confusion with the state or archipelago. The uninhabited Northwestern Hawaiian Islands make up most of the Papahānaumokuākea Marine National Monument, the United States' largest protected area and the fourth-largest in the world. Of the 50 U.S. states, Hawaii is the eighth-smallest in land area and the 11th-least populous, but with 1.4 million residents ranks 13th in population density. Two-thirds of the population lives on O'ahu, home to the state's capital and largest city, Honolulu. Hawaii is among the country's most diverse states, owing to its central location in the Pacific and over two centuries of migration. As one of only six majority-minority states, it has the country's only Asian American plurality, its largest Buddhist community, and the largest proportion of multiracial people. Consequently, it is a unique melting pot of North American and East Asian cultures, in addition to its indigenous Hawaiian heritage. Settled by Polynesians some time between 1000 and 1200 CE, Hawaii was home to numerous independent chiefdoms. In 1778, British explorer James Cook was the first known non-Polynesian to arrive at the archipelago; early British influence is reflected in the state flag, which bears a Union Jack. An influx of European and American explorers, traders, and whalers arrived shortly after leading to the decimation of the once isolated Indigenous community by introducing diseases such as syphilis, gonorrhea, tuberculosis, smallpox, measles, leprosy, and typhoid fever, reducing the native Hawaiian population from between 300,000 and one million to less than 40,000 by 1890. Hawaii became a unified, internationally recognized kingdom in 1810, remaining independent until American and European businessmen overthrew the monarchy in 1893; this led to annexation by the U.S. in 1898. As a strategically valuable U.S. territory, Hawaii was attacked by Japan on December 7, 1941, which brought it global and historical significance, and contributed to America's decisive entry into World War II. Hawaii is the most recent state to join the union, on August 21, 1959. In 1993, the U.S. government formally apologized for its role in the overthrow of Hawaii's government, which spurred the Hawaiian sovereignty movement.'''

In [82]:
print(old_summarization_pipeline(text))

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [70]:
text = "Hawaii comprises nearly the entire Hawaiian archipelago, 137 volcanic islands spanning 1,500 miles (2,400 km) that are physiographically and ethnologically part of the Polynesian subregion of Oceania. The state's ocean coastline is consequently the fourth-longest in the U.S., at about 750 miles (1,210 km).[b] The eight main islands, from northwest to southeast, are Niʻihau, Kauaʻi, Oʻahu, Molokaʻi, Lānaʻi, Kahoʻolawe, Maui, and Hawaiʻi—the last of these, after which the state is named, is often called the "Big Island" or "Hawaii Island" to avoid confusion with the state or archipelago. The uninhabited Northwestern Hawaiian Islands make up most of the Papahānaumokuākea Marine National Monument, the United States' largest protected area and the fourth-largest in the world. Of the 50 U.S. states, Hawaii is the eighth-smallest in land area and the 11th-least populous, but with 1.4 million residents ranks 13th in population density. Two-thirds of the population lives on O'ahu, home to the state's capital and largest city, Honolulu. Hawaii is among the country's most diverse states, owing to its central location in the Pacific and over two centuries of migration. As one of only six majority-minority states, it has the country's only Asian American plurality, its largest Buddhist community, and the largest proportion of multiracial people. Consequently, it is a unique melting pot of North American and East Asian cultures, in addition to its indigenous Hawaiian heritage. Settled by Polynesians some time between 1000 and 1200 CE, Hawaii was home to numerous independent chiefdoms. In 1778, British explorer James Cook was the first known non-Polynesian to arrive at the archipelago; early British influence is reflected in the state flag, which bears a Union Jack. An influx of European and American explorers, traders, and whalers arrived shortly after leading to the decimation of the once isolated Indigenous community by introducing diseases such as syphilis, gonorrhea, tuberculosis, smallpox, measles, leprosy, and typhoid fever, reducing the native Hawaiian population from between 300,000 and one million to less than 40,000 by 1890. Hawaii became a unified, internationally recognized kingdom in 1810, remaining independent until American and European businessmen overthrew the monarchy in 1893; this led to annexation by the U.S. in 1898. As a strategically valuable U.S. territory, Hawaii was attacked by Japan on December 7, 1941, which brought it global and historical significance, and contributed to America's decisive entry into World War II. Hawaii is the most recent state to join the union, on August 21, 1959. In 1993, the U.S. government formally apologized for its role in the overthrow of Hawaii's government, which spurred the Hawaiian sovereignty movement."

In [71]:
old_summarization_pipeline(text)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
predictor.predict({
    "parameters": {'truncation': True},
	'inputs': '''Hawaii comprises nearly the entire Hawaiian archipelago, 137 volcanic islands spanning 1,500 miles (2,400 km) that are physiographically and ethnologically part of the Polynesian subregion of Oceania. The state's ocean coastline is consequently the fourth-longest in the U.S., at about 750 miles (1,210 km).[b] The eight main islands, from northwest to southeast, are Niʻihau, Kauaʻi, Oʻahu, Molokaʻi, Lānaʻi, Kahoʻolawe, Maui, and Hawaiʻi—the last of these, after which the state is named, is often called the "Big Island" or "Hawaii Island" to avoid confusion with the state or archipelago. The uninhabited Northwestern Hawaiian Islands make up most of the Papahānaumokuākea Marine National Monument, the United States' largest protected area and the fourth-largest in the world. Of the 50 U.S. states, Hawaii is the eighth-smallest in land area and the 11th-least populous, but with 1.4 million residents ranks 13th in population density. Two-thirds of the population lives on O'ahu, home to the state's capital and largest city, Honolulu. Hawaii is among the country's most diverse states, owing to its central location in the Pacific and over two centuries of migration. As one of only six majority-minority states, it has the country's only Asian American plurality, its largest Buddhist community, and the largest proportion of multiracial people. Consequently, it is a unique melting pot of North American and East Asian cultures, in addition to its indigenous Hawaiian heritage. Settled by Polynesians some time between 1000 and 1200 CE, Hawaii was home to numerous independent chiefdoms. In 1778, British explorer James Cook was the first known non-Polynesian to arrive at the archipelago; early British influence is reflected in the state flag, which bears a Union Jack. An influx of European and American explorers, traders, and whalers arrived shortly after leading to the decimation of the once isolated Indigenous community by introducing diseases such as syphilis, gonorrhea, tuberculosis, smallpox, measles, leprosy, and typhoid fever, reducing the native Hawaiian population from between 300,000 and one million to less than 40,000 by 1890. Hawaii became a unified, internationally recognized kingdom in 1810, remaining independent until American and European businessmen overthrew the monarchy in 1893; this led to annexation by the U.S. in 1898. As a strategically valuable U.S. territory, Hawaii was attacked by Japan on December 7, 1941, which brought it global and historical significance, and contributed to America's decisive entry into World War II. Hawaii is the most recent state to join the union, on August 21, 1959. In 1993, the U.S. government formally apologized for its role in the overthrow of Hawaii's government, which spurred the Hawaiian sovereignty movement.'''
})

## Import data

In [11]:
role = sagemaker.get_execution_role()
data_location = 's3://datasets/text_summarization/corpus/corpus(cleaned).csv'

df = pd.DataFrame(pd.read_csv(data_location))

In [22]:
%%time

summarized = pd.DataFrame()

for i in range(5):
    data = {
        "inputs": df["TEXT"][i],
        "parameters": {'truncation': True}
    }
    summarized.at[i, "TEXT"] = df["TEXT"][i]
    summarized.at[i, "SUMMARIZED"] = predictor.predict(data)[0]['summary_text']
    if i % 10 == 0:
        wr.s3.to_csv(
            df=summarized,
            path='s3://datasets/text_summarization/summarized/facebook_bart-large-cnn/bart-large-cnn(trained).csv'
        )
        print(i)
    
summarized

0


ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "{
  "code": 400,
  "type": "InternalServerException",
  "message": "index out of range in self"
}
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/huggingface-pytorch-inference-2022-10-19-23-20-36-357 in account 047476809233 for more information.

---
## Use model with the [pipeline API](https://huggingface.co/docs/transformers/main_classes/pipelines)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("bart-large-cnn")

model = AutoModelForSeq2SeqLM.from_pretrained("bart-large-cnn")

In [ ]:
!pip install ipywidgets

In [26]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model= "facebook/bart-large-cnn")

In [ ]:
ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""
print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))